### Installs & Imports

# 1. Named Entity Recognition

#### Training & Validation Sets

In [ ]:
# currently the instances are sentences, we need them to be transformed so that the instances are words
# also when transforming add sentence_id and token_id to the words within the same sentence 
# need to merge the training files into one dataframe, we can also include the test or the validation in the training data 
# and keep only either test and validation seperately for checking 

#### Test Set

#### Feature Engineering

#### Model Training

# 2. Sentiment Analysis

#### Training & Validation Sets

#### Test Set (the same test set is used for both sentiment analysis and topic modeling)

#### Feature Engineering

#### Model Training

# 3. Topic Classification

#### Training & Validation Sets

In [ ]:
# perhaps filter reviews that are one-word like "good", "great" 
# format the reviews text the same way, some of them are within "", others are not, make all without ""

#### Test Set (the same test set is used for both sentiment analysis and topic modeling)

#### Feature Engineering

#### Model Training